In [1]:
import torch
from rich.markdown import Markdown

device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
%cd ..

/home/oscarn/flan-gpt2


In [3]:
from models.model_utils import GPT2Model

#model_name = "gpt2-multitask-4_V2"
#model_name = "flan-gpt2-medium-distill_V2"
#model_name = "gpt2-multitask_V3"
model_name = "output/flan-gpt2-medium-distill_V3"
#model_name = "pythia-multitask_V3"

#model_name = "OscarNav/flan-gpt2-medium-ft"

wrapped_model = GPT2Model(model_name, device)
model = wrapped_model.get_model()
tokenizer = wrapped_model.get_tokenizer()

output/flan-gpt2-medium-distill_V3


In [13]:
from models.model_utils import GPT2Model

wrapped_model = GPT2Model("output/gpt2-lora", device, peft=True)
peft_model = wrapped_model.get_model(peft=True)
peft_tokenizer = wrapped_model.get_tokenizer()

openai-community/gpt2-medium


In [3]:
from models.model_utils import SmolLMModel, T5Model

#model_name = "output/flan-SmolLM-135M-distill"
#model_name = "output/flan-flan-t5-base-ft"
model_name = "OscarNav/flan-flan-t5-large-ft"

wrapped_model = T5Model(model_name, device)
model = wrapped_model.get_model()
model.resize_token_embeddings(len(wrapped_model.get_tokenizer()))
tokenizer = wrapped_model.get_tokenizer()

config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [14]:
from eval.eval_utils import Evaluation
from data.data_utils import create_instruct_dataset


eval = Evaluation(peft_model, peft_tokenizer, device)

In [15]:
num_samples = 100

In [16]:
template = """
## INPUT PROMPT
{prompt}
## GROUND_TRUTH
{completion}
## MODEL OUTPUT
{prediction}
"""

### ANLI

In [17]:
dataset = create_instruct_dataset(num_samples, ["anli"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/16946 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16946 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  7.00it/s]


INPUT PROMPT                                                    

L. M. Singhvi (9 November 1931 – 6 October 2007) was an Indian jurist, parliamentarian, scholar, writer and        
diplomat. He was, after V. K. Krishna Menon, the second-longest-serving High Commissioner for India in the United  
Kingdom (1991–97) He was conferred Padma Bhushan in 1998.                                                          

Based on that paragraph can we conclude that this sentence is true? L.M. Singvhi served 76 years.                  

OPTIONS:                                                                                                           

 • entailment                                                                                                      
 • neutral                                                                                                         
 • contradiction                                                                                                   


                                                   GROUND_TRUTH                                                    

neutral                                                                                                            


                                                   MODEL OUTPUT                                                    

L. M. Singhvi (9 November 1931 – 6 October 2007) was an Indian jurist, parliamentarian, scholar, writer and        
diplomat. He was, after V. K. Krishna Menon, the second-longest-serving High Commissioner for India in the United  
Kingdom (1991–97) He was conferred Padma Bhushan in 1998.                                                          

Based on that paragraph can we conclude that this sentence is true? L.M. Singvhi served 76 years.                  

OPTIONS:                                                                                                           

 • entailment                                                                                                      
 • neutral                                                                                                         
 • contradictionneutral

## Common Gen

In [18]:
dataset = create_instruct_dataset(num_samples, ["common_gen"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.19it/s]


INPUT PROMPT                                                    

Here are some concepts: ['grass', 'horse', 'eat']                                                                  

What is a sentence about these concepts?                                                                           


                                                   GROUND_TRUTH                                                    

The horses are eating grass.                                                                                       


                                                   MODEL OUTPUT                                                    

Here are some concepts: ['grass', 'horse', 'eat']                                                                  

What is a sentence about these concepts?                                                                           

He was eating grass

## SQUAD

In [19]:
dataset = create_instruct_dataset(num_samples, ["squad"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).


Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/87241 [00:00<?, ? examples/s]

Filter:   0%|          | 0/87241 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:10<00:00, 10.30s/it]


INPUT PROMPT                                                    

What is the title of this article:                                                                                 

Father Joseph Carrier, C.S.C. was Director of the Science Museum and the Library and Professor of Chemistry and    
Physics until 1874. Carrier taught that scientific research and its promise for progress were not antagonistic to  
the ideals of intellectual and moral culture endorsed by the Church. One of Carrier's students was Father John     
Augustine Zahm (1851–1921) who was made Professor and Co-Director of the Science Department at age 23 and by 1900  
was a nationally prominent scientist and naturalist. Zahm was active in the Catholic Summer School movement, which 
introduced Catholic laity to contemporary intellectual issues. His book Evolution and Dogma (1896) defended certain
aspects of evolutionary theory as true, and argued, moreover, that even the great Church teachers Thomas Aquinas   
and Augustine taught something like it. The intervention of Irish American Catholics in Rome prevented Zahm's      
censure by the Vatican. In 1913, Zahm and former President Theodore Roosevelt embarked on a major expedition       
through the Amazon.                                                                                                


                                                   GROUND_TRUTH                                                    

University_of_Notre_Dame                                                                                           


                                                   MODEL OUTPUT                                                    

What is the title of this article:                                                                                 

Father Joseph Carrier, C.S.C. was Director of the Science Museum and the Library and Professor of Chemistry and    
Physics until 1874. Carrier taught that scientific research and its promise for progress were not antagonistic to  
the ideals of intellectual and moral culture endorsed by the Church. One of Carrier's students was Father John     
Augustine Zahm (1851–1921) who was made Professor and Co-Director of the Science Department at age 23 and by 1900  
was a nationally prominent scientist and naturalist. Zahm was active in the Catholic Summer School movement, which 
introduced Catholic laity to contemporary intellectual issues. His book Evolution and Dogma (1896) defended certain
aspects of evolutionary theory as true, and argued, moreover, that even the great Church teachers Thomas Aquinas   
and Augustine taught something like it. The intervention of Irish American Catholics in Rome prevented Zahm's      
censure by the Vatican. In 1913, Zahm and former President Theodore Roosevelt embarked on a major expedition       
through the Amazon. At that time, though, the scientific world did not seem fully invested in the "evolutionary    
solution" advocated by the Church. In 1915, at the age of 77, Joseph Carrier died in Chicago of a respiratory      
disease.                                                                                                           

The author writes:                                                                                                 

John Augustine Zahm.                                                                                               

Born in 1851, Joseph Carrier (1851-1921) became renowned as Director of the Royal Society of Chemistry and         
Professor of Chemistry at St Mary's College, University of Oxford.                                                 

[…] In 1913, he published Evolution and Dogma which defended some aspects of evolutionary theory as true, and      
argued, moreover, that even the great Church fathers Thomas Aquinas and Augustine taught something like it. The    
intervention of Irish Americans in Rome prevented his censure by the Vatican, and the Church r

## Cosmos QA

In [12]:
dataset = create_instruct_dataset(num_samples, ["cosmos_qa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.


Map:   0%|          | 0/25262 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25262 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:06<00:00,  6.31s/it]


INPUT PROMPT                                                    

I learned new generation web technologies ASP hopefully looking forward for the day I make my homepage in aspx . I 
moved to college hostel this year . I do nt know why I spent so much time infront of my computer and I think its   
time to change my character . Not going to happen in this situation any more and I guess more time and try is      
required ; ) this blog ends with no normal start and no normal end .                                               

Question: Why ca n't I stop spending so much time in front of the computer to change my character ? OPTIONS:       

 • None of the above choices .                                                                                     
 • I 'm going to college for computer sciences , so I have to play on the computer a lot .                         
 • I 'm going to college for blogging , so I have to work on the computer a lot .                                  
 • I 'm going to college for computer programming , so I have to work on the computer a lot .                      


                                                   GROUND_TRUTH                                                    

I 'm going to college for computer programming , so I have to work on the computer a lot .                         


                                                   MODEL OUTPUT                                                    

I learned new generation web technologies ASP hopefully looking forward for the day I make my homepage in aspx. I  
moved to college hostel this year. I do nt know why I spent so much time infront of my computer and I think its    
time to change my character. Not going to happen in this situation any more and I guess more time and try is       
required ; ) this blog ends with no normal start and no normal end.                                                

Question: Why can't I stop spending so much time in front of the computer to change my character? OPTIONS:         

 • None of the above choices.                                                                                      
 • I'm going to college for computer sciences, so I have to play on the computer a lot.                            
 • I'm going to college for blogging, so I have to work on the computer a lot.                                     
 • I'm going to college for computer programming, so I have to work on the computer a lot.                         
 • I'm going to college for learning languages, so I have to study for several hours.                              
 • I am going to college for teaching computer science, so I only study in college part time when I am learning.   
 • I am going to college for teaching programming, so I study and also write blog posts in other subject.          
 • My family is going to university for post-graduation and I have to study for years.                             
 • My sister/uncle is going to university for post-graduation and I have to study for years.                       
 • Maybe I can go back then.                                                                                       
 • I understand the limitations of the internet and how my parents and everyone else in the world are going to use 
   it. I know how to fix many things on the internet but it is hard.                                               
 • Maybe I have to do a lot of computer coding.                                                                    
 • I am going to college for programming, so I'm going to study

## CoQA

In [12]:
dataset = create_instruct_dataset(num_samples, ["coqa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=150, return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/7199 [00:00<?, ? examples/s]

Map:   0%|          | 0/4063 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4063 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:08<00:00,  8.72s/it]


INPUT PROMPT                                                    

Answer the questions at the end based on the text.                                                                 

Ibn Rushd (; 14 April 1126 – 10 December 1198), full name (), often Latinized as Averroes (), was a medieval       
Andalusian polymath. He wrote on logic, Aristotelian and Islamic philosophy, theology, the Maliki school of Islamic
jurisprudence, psychology, political and Andalusian classical music theory, geography, mathematics, and the        
mediæval sciences of medicine, astronomy, physics, and celestial mechanics. Ibn Rushd was born in Córdoba, Al      
Andalus (present-day Spain), and died at Marrakesh in present-day Morocco. His body was interred in his family tomb
at Córdoba. The 13th-century philosophical movement in Latin Christian and Jewish tradition based on Ibn Rushd's   
work is called Averroism.                                                                                          

Ibn Rushd was a defender of Aristotelian philosophy against Ash'ari theologians led by Al-Ghazali. Although highly 
regarded as a legal scholar of the Maliki school of Islamic law, Ibn Rushd's philosophical ideas were considered   
controversial in Ash'arite Muslim circles. Whereas al-Ghazali believed that any individual act of a natural        
phenomenon occurred only because God willed it to happen, Ibn Rushd insisted phenomena followed natural laws that  
God created.                                                                                                       

Ibn Rushd had a greater impact on Christian Europe, being known by the "the Commentator" for his detailed          
emendations to Aristotle. Latin translations of Ibn Rushd's work led the way to the popularization of Aristotle.   

  1 Where was Rushd born?                                                                                          
  2 In what country is this today?                                                                                 
  3 Where did he die?                                                                                              
  4 In what modern place?                                                                                          
  5 Where was he laid to rest?                                                                                     
  6 Who was Rushd?                                                                                                 
  7 What is one subject he wrote on?                                                                               
  8 And another?                                                                                                   
  9 And another?                                                                                                   
 10 Did he also write about the study of stars?                                                                    
 11 When was he born?                                                                                              
 12 When did he die?                                                                                               
 13 What did he defend?                                                                                            
 14 Against whom?                                                                                                  
 15 Who was their leader?                                                                                          
 16 By what title was Rushd known in Europe?                                                                       
 17 For what?                                                                                                      
 18 Did his work make Aristotle better known?                                                                      
 19 True or False: Rushd believed everything happened by the direct will of God.                      

## XSum

In [13]:
dataset = create_instruct_dataset(num_samples, ["xsum"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:15<00:00, 15.09s/it]


INPUT PROMPT                                                    

Write an article based on this "Aston Villa manager Remi Garde refused to talk about his future following his      
side's 2-0 defeat at West Ham."                                                                                    


                                                   GROUND_TRUTH                                                    

The Midlands club are bottom of the Premier League - 10 points from safety - and failed to make any signings in the
January transfer window. Reports have since suggested Garde will reassess his future in the summer. But the        
Frenchman said: "I'm speaking only about the game. I have nothing else to say. It's not the time to answer this    
question." Villa had a penalty appeal rejected against the Hammers before having Jordan Ayew sent off after just 17
minutes. West Ham opened the scoring through Michail Antonio - a goal Garde thought might have been offside -      
before Cheikhou Kouyate added a second for the Upton Park outfit. "I'm not going to speak any more (about) the     
incident, it's a major one in the game but I don't have to explain my feelings, and you can guess Jordan's         
feelings," he added. "Whatever happens in the dressing room is private. "But I'm not sure it was the only incident 
that turned the game. Of course I think we should have had a penalty. "I'm not a lucky or unlucky man, but I don't 
like to hear it's because we are at the bottom. The rules in football are there for everybody."                    


                                                   MODEL OUTPUT                                                    

He only came on as a substitute after Villa were sacked with less than a month remaining. "The most important thing
for me is to continue my work in this club," Garde told BBC Radio Stoke. "The next thing I'll talk about is my     
future but it just goes on and on, like the next player who comes in and says to me that he is prepared to take    
more football clubs." Garde had already confirmed he was in the side to face Villa IF in the FA Cup match on Friday
night against Southampton. The Portuguese was a key player who helped Villa to win the first five points over      
Southampton in the Champions League on Tuesday and made his first appearance as part of the side's five-goal       
victory. He was replaced at City before being put back in charge of their game with West Ham just six days before  
Garde's departure from Lazio. Garde came to the club from Southampton in June when he

## BoolQ

In [14]:
dataset = create_instruct_dataset(num_samples, ["bool_q"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/9399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9399 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  4.65it/s]


INPUT PROMPT                                                    

The Bucks have won one league title (1971), two conference titles (1971 and 1974), and 13 division titles          
(1971--1974, 1976, 1980--1986, 2001). They have featured such notable players as Kareem Abdul-Jabbar, Sidney       
Moncrief, Oscar Robertson, Bob Dandridge, Bob Lanier, Glenn Robinson, Ray Allen, Sam Cassell, Junior Bridgeman,    
Michael Redd, Terry Cummings, Vin Baker, Jon McGlocklin, Marques Johnson, and Brian Winters. Answer this question, 
making sure that the answer is supposed by the text: have the milwaukee bucks ever won a championship?             

OPTIONS:                                                                                                           

 • True                                                                                                            
 • False                                                                                                           


                                                   GROUND_TRUTH                                                    

True                                                                                                               


                                                   MODEL OUTPUT                                                    

True

## Python Code

In [15]:
dataset = create_instruct_dataset(num_samples, ["python_code"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=1000, return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/16750 [00:00<?, ? examples/s]

Map:   0%|          | 0/16237 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16237 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:08<00:00,  8.63s/it]


INPUT PROMPT                                                    

Create a Python algorithm to identify the longest word in a sentence. “The rain in Spain stays mainly in the       
plain.”                                                                                                            


                                                   GROUND_TRUTH                                                    

                                                                                                                   
 def find_longest_word(sentence):                                                                                  
     words = sentence.split()                                                                                      
     # initialise longest_word variable with 0                                                                     
     longest_word = 0                                                                                              
     for word in words:                                                                                            
         if len(word) > longest_word:                                                                              
             longest_word = len(word)                                                                              
             longest_word_str = word                                                                               
     return longest_word_str                                                                                       
                                                                                                                   


                                                   MODEL OUTPUT                                                    

python import grammatical_node # symlink between long and short word symlink.extend(symlink_word) def              
longest_word(s): longest_word = 'The rain in Spain stays mostly in the plain' answer = [longest_word in s if s[j]!=
'in' else shortest_word in symlink) longest_word = shortest_word

## Eng-Spa

In [18]:
dataset = create_instruct_dataset(num_samples, ["eng_spa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


INPUT PROMPT                                                    

Translate from English to Spanish:                                                                                 

Can I keep it?                                                                                                     


                                                   GROUND_TRUTH                                                    

¿Puedo quedármelo?                                                                                                 


                                                   MODEL OUTPUT                                                    

Me puedo mantenerla?

## PAWS

In [19]:
dataset = create_instruct_dataset(num_samples, ["paws"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  4.65it/s]


INPUT PROMPT                                                    

Brockton is approximately 25 miles northeast of Providence , Rhode Island , and 30 miles south of Boston .         

Brockton is located approximately 25 miles northeast of Providence , Rhode Island and 30 miles south of Boston .   

Do the above sentences mean the same thing? OPTIONS:                                                               

 • No                                                                                                              
 • Yes                                                                                                             


                                                   GROUND_TRUTH                                                    

Yes                                                                                                                


                                                   MODEL OUTPUT                                                    

Yes

## Quora

In [21]:
dataset = create_instruct_dataset(num_samples, ["quora"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/56402 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


INPUT PROMPT                                                    

How much longer can the Russian economy sustain its war with Ukraine?                                              


                                                   GROUND_TRUTH                                                    

Much much longer than the collapsing western economies                                                             


                                                   MODEL OUTPUT                                                    

The only thing that can stop this war will be a collapse of the economy.

## Alpaca

In [24]:
dataset = create_instruct_dataset(num_samples, ["alpaca"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=True)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/20100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


INPUT PROMPT                                                    

Search the web and find the title of the longest novel.                                                            


                                                   GROUND_TRUTH                                                    

The title of the longest novel is "Gordale", written by Carolyn Redfearn.                                          


                                                   MODEL OUTPUT                                                    

Taking Away the Dead / The Longest Book Ever Written